Loads JSON files storing cricket match data to a staging database

In [1]:
import pandas as pd
import json
import os
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import time

In [2]:
# Path to your JSON files
JSON_DIR = 'C:\\Users\\chris\\cricket_json'
SERVER = 'DESKTOP-IIHCV0F\SQLEXPRESS'
DATABASE = 'CricketDBStaging'
DRIVER = 'ODBC Driver 17 for SQL Server' 

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\chris\AppData\Local\Temp\ipykernel_8088\1615733251.py:3: SyntaxWarning: invalid escape sequence '\S'
  SERVER = 'DESKTOP-IIHCV0F\SQLEXPRESS'


In [3]:
# Function to insert match details to staging tables
def load_match_to_sql(cursor, filename, match):

    
    match_info = match['info']


    match_win_type = None
    match_win_type = None
    match_win_margin = None

    if 'city' in match_info:
        match_city = match_info['city']
    else:
        match_city = 'Unknown'

    if 'event' in match_info:
        event_name = match_info['event']['name']
    else:
        event_name = None


    match_dates = match_info['dates']
    match_type = match_info['match_type']

    if 'result' in match_info['outcome']:
        match_outcome = match_info['outcome']['result']
        match_win_type = None
        match_winner = None
        match_win_type = None
        match_win_margin = None
    else:
        match_outcome = 'result'

    team1 = match_info['teams'][0]
    team2 = match_info['teams'][1]

    if 'winner' in match_info['outcome']:
        match_winner = match_info['outcome']['winner']

        if 'by' in match_info['outcome']:
        
            if 'runs' in match_info['outcome']['by']:
                match_win_type = 'runs'
                match_win_margin = match_info['outcome']['by']['runs']

            if 'innings' in match_info['outcome']['by']:
                match_win_type = 'innings'
                match_win_margin = match_info['outcome']['by']['innings']

            if 'wickets' in match_info['outcome']['by']:
                match_win_type = 'wickets'
                match_win_margin = match_info['outcome']['by']['wickets']
        else:
            match_win_type = None
            match_win_margin = None
    else:
        match_winner = None

    team1_players = match_info['players'][team1]
    team2_players = match_info['players'][team2]

    match_innings = match['innings']

    # Insert a single row into match table
    cursor.execute("""
        INSERT INTO stg_matches (match_date, city, match_type, result, team1, team2, winning_team, win_type, win_margin, event_name, team1_players, team2_players, source_filename) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (json.dumps(match_dates), match_city, match_type, match_outcome, team1, team2, match_winner, match_win_type, match_win_margin, event_name, json.dumps(team1_players), json.dumps(team2_players), filename))

    cursor.execute("""SELECT IDENT_CURRENT('stg_matches')""")
    match_id = cursor.fetchone()[0]

    # Insert each innings
    for idx, innings in enumerate(match_innings):
        innings_no = idx + 1
        innings_team = innings['team']

        if 'overs' in innings:
            overs = innings['overs']
        else:
            overs = None

        cursor.execute("""
            INSERT INTO stg_innings (match_id, innings_number, team) 
            VALUES (?, ?, ?);
        """, (match_id, innings_no, innings_team))

        cursor.execute("""SELECT IDENT_CURRENT('stg_innings')""")
        innings_id = cursor.fetchone()[0]
    
        if overs is not None:
            # Insert each over
            for over in overs:
                over_number = over['over']
                deliveries = over['deliveries']
                    
                cursor.execute("""
                    INSERT INTO stg_overs (innings_id, over_number) 
                    VALUES (?, ?);
                """, (innings_id, over_number))

                cursor.execute("""SELECT IDENT_CURRENT('stg_overs')""")
                over_id = cursor.fetchone()[0]

                # Insert each delivery
                for idy, delivery in enumerate(deliveries):
                    delivery_number = idy + 1

                    batter = delivery['batter']
                    bowler = delivery['bowler']
                    non_striker = delivery['non_striker']

                    total_runs = delivery['runs']['total']
                    extras = delivery['runs']['extras']
                    batter_runs = delivery['runs']['batter']

                    if 'wickets' in delivery:
                        wicket_taken = 1
                    else:
                        wicket_taken = 0

                    cursor.execute("""
                        INSERT INTO stg_deliveries (over_id, batter, bowler, non_striker, delivery_number, total_runs, batter_runs, extras, wicket_taken) 
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
                    """, (over_id, batter, bowler, non_striker, delivery_number, total_runs, batter_runs, extras, wicket_taken))

                    cursor.execute("""SELECT IDENT_CURRENT('stg_deliveries')""")
                    delivery_id = cursor.fetchone()[0]


                    if wicket_taken:
                        for wicket in delivery['wickets']:
                            player_out = wicket['player_out']
                            kind = wicket['kind']

                            cursor.execute("""
                                INSERT INTO stg_wickets (delivery_id, player_out, kind) 
                                VALUES (?, ?, ?)
                            """, (delivery_id, player_out, kind))

# Function to load JSON data from a file
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Process each JSON file in the directory
def load_all_json_files(directory):
    all_data = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            data = load_json(file_path)
            # Assuming the filename (without extension) is used as the key
            filename = os.path.splitext(filename)[0]
            all_data.append((filename, data))
    return all_data

In [4]:
# Load JSON data
matches = load_all_json_files(JSON_DIR)

print(len(matches))

# Get subset of matches for testing
#matches = matches[0:1000]

17735


In [5]:
conn = pyodbc.connect(f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;')
cursor = conn.cursor()

for idx, match in enumerate(matches):
    print("Processing: " + str(idx))
    start = time.time()

    load_match_to_sql(cursor, match[0], match[1])

    end = time.time()

    print ("Elapsed time: " + str(end-start))

cursor.close
conn.commit()
conn.close()

Processing: 0
Elapsed time: 3.5255205631256104
Processing: 1
Elapsed time: 0.8361914157867432
Processing: 2
Elapsed time: 1.8719327449798584
Processing: 3
Elapsed time: 3.357283353805542
Processing: 4
Elapsed time: 1.936422348022461
Processing: 5
Elapsed time: 1.743995189666748
Processing: 6
Elapsed time: 0.8520324230194092
Processing: 7
Elapsed time: 0.7699794769287109
Processing: 8
Elapsed time: 0.6583602428436279
Processing: 9
Elapsed time: 0.5962152481079102
Processing: 10
Elapsed time: 0.4028592109680176
Processing: 11
Elapsed time: 0.4334831237792969
Processing: 12
Elapsed time: 0.3946042060852051
Processing: 13
Elapsed time: 0.2846715450286865
Processing: 14
Elapsed time: 0.7978942394256592
Processing: 15
Elapsed time: 0.4857304096221924
Processing: 16
Elapsed time: 0.2296457290649414
Processing: 17
Elapsed time: 0.44763803482055664
Processing: 18
Elapsed time: 0.418414831161499
Processing: 19
Elapsed time: 0.46846461296081543
Processing: 20
Elapsed time: 0.3388986587524414
Proc